In [4]:
import pydotplus as pydot
from py2neo import Graph

def visualize_neo4j_schema(neo4j_url, neo4j_user, neo4j_password, output_image_path):
    # Connect to Neo4j
    graph = Graph(neo4j_url, auth=(neo4j_user, neo4j_password))

    # Retrieve Neo4j schema information
    schema_query = """
    CALL db.schema.visualization()
    YIELD nodes, relationships
    RETURN nodes, relationships
    """
    result = graph.run(schema_query).data()[0]

    # Create a directed graph using pydotplus
    graphviz_graph = pydot.Dot(graph_type='digraph')

    # Add nodes to the graph
    for node in result['nodes']:
        # Check if 'labels' attribute is present and not None
        if 'labels' in node and node['labels'] is not None and len(node['labels']) > 0:
            node_label = node['labels'][0]
            graphviz_graph.add_node(pydot.Node(node_label, shape='box', color='blue'))

    # Add relationships to the graph
    for relationship in result['relationships']:
        # Check if 'start' and 'end' are present and not None
        if 'start' in relationship and 'end' in relationship and relationship['start'] is not None and relationship['end'] is not None:
            start_label = relationship['start']['labels'][0] if 'labels' in relationship['start'] and relationship['start']['labels'] else None
            end_label = relationship['end']['labels'][0] if 'labels' in relationship['end'] and relationship['end']['labels'] else None
            relationship_type = relationship['type'] if 'type' in relationship else None

            # Check if required attributes are present and not None
            if start_label is not None and end_label is not None and relationship_type is not None:
                # Assuming a directed graph for simplicity
                graphviz_graph.add_edge(pydot.Edge(start_label, end_label, label=relationship_type, color='red'))

    # Save the graph as an image
    graphviz_graph.write_png(output_image_path)

if __name__ == "__main__":
    neo4j_url = "bolt://localhost:7687"
    neo4j_user = "neo4j"
    neo4j_password = "LogisticData"
    output_image_path = "neo4j_schema.png"

    visualize_neo4j_schema(neo4j_url, neo4j_user, neo4j_password, output_image_path)


InvocationException: GraphViz's executables not found